# TAREA 2

### Limpieza de datos

Como se denotó en la tarea anterior los datos a utilizar son los correspondientes al Inventario Nacional de Emisiones de Gases y Compuestos de Efecto Invernadero del INECC que se puede encontrar en la siguiente liga: https://datos.gob.mx/busca/dataset/inventario-nacional-de-emisiones-de-gases-y-compuestos-de-efecto-invernadero-inegycei.

De la liga se usan un total de 2 archivos:
* Inventario Nacional de Emisiones de Gases y Compuestos de Efecto Invernadero (INEGYCEI) 1990-2019
* Inventario Nacional de Emisiones de Gases y Compuestos de Efecto Invernadero INEGYCEI 2020-2021

Ambos archivos de Excel tienen formatos que generan conflictos al momento de importarlos por lo que se usó la pestaña de *Automatización* y la función de *Grabar acciones* de Excel para generar un script para copiar solo los datos de las hojas a hojas de trabajo nuevas:

``````
function main(workbook: ExcelScript.Workbook) {
	// Add a new worksheet
	let hoja3 = workbook.addWorksheet();
	let selectedSheet = workbook.getActiveWorksheet();
	// Paste to range A1 on hoja3 from range A2:AF190 on selectedSheet
	hoja3.getRange("A1").copyFrom(selectedSheet.getRange("A2:AF190"), ExcelScript.RangeCopyType.values, false, false);
	// Delete range 1:1 on hoja3
	hoja3.getRange("1:1").delete(ExcelScript.DeleteShiftDirection.up);
}
``````
Una vez generadas las nuevas hojas de datos se les cambia el nombre y se unen todas en un solo archivo al que se le llamó ***solo_valores_1990_2021***.

Se importan las librerías que se necesitarán

In [1]:
import pandas as pd
import numpy as np
import math

Leer el archivo y guardar las diferentes hojas en un dataframe distinto

In [2]:
archivo = 'C:/Users/patty/Documents/Maestria Base de Datos/Aprendizaje_Automatico_priv/solo_valores_1990_2021.xlsx'

df = {}
for i in range(1990,2022):
    j = str(i)+'.2'
    df[i] = pd.read_excel(archivo, sheet_name= j)

Para esto se crea un diccionario **df** vacío. Posteriormente se inicia un ciclo para asignar una hoja distinta a cada elemento del diccionario.

Se ha de notar que el ciclo se hizo en el rango de 1990 a 2022 para después convertir dicho número a cadena y agregar *'.2'* ya que esos son los nombres que tienen las hojas en el libro de Excel.

Una vez terminado podemos mandar a llamar un dataframe para revisar los datos:

In [3]:
df[1990]

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,0,0,0,0,0,36.517125,467413.123797,466720.011075,NaN,77.5572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Aviación internacional,2052.937,0.396,14.985,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2068.318,NaN,NaN,NaN
177,Marítimo internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Emisiones de CO2 por quema de biomasa,35955.863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35955.863,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Revisamos que todos los dataframes tengan el mismo tamaño con la ayuda de la función *shape*:

In [4]:
c = True

for i in range(1990,2021):
    if df[i].shape != df[i+1].shape:
        c = False
c

False

Al hacer una pequeño cambio al ciclo podemos saber a partir de que año cambia el tamaño de los data frame:

In [5]:
c = True

for i in range(1990,2021):
    if df[i].shape != df[i+1].shape:
        c = False
        y=i
        break
y

2019

Comparamos los tamaños de cada dataframe para saber donde varían:

In [6]:
df[2019].shape

(181, 32)

In [7]:
df[2020].shape

(187, 32)

In [8]:
df[2021].shape

(187, 32)

Podemos ver que la diferencia es en el número de filas y no en las columnas.

Para poder ver las diferencias en las columnas podemos crear un diccionario con la primera columna de cada dataframe para comparar donde son diferentes:

In [9]:
m= {}

for i in range(0,181):
   m[i]= [df[2019].iloc[i,0] , df[2020].iloc[i,0]]

for i in range(181,187):
   m[i]=['-',df[2020].iloc[i,0]]
m


{0: [nan, nan],
 1: [nan, nan],
 2: ['  Potencial de calentamiento', '  Potencial de calentamiento'],
 3: [nan, nan],
 4: ['EMISIONES NETAS (Gg de CO2e)', 'EMISIONES NETAS (Gg de CO2e)'],
 5: ['[1] Energía', '[1] Energía'],
 6: ['[1A] Actividades de quema del combustible',
  '[1A] Actividades de quema del combustible'],
 7: ['[1A1] Industrias de la energía', '[1A1] Industrias de la energía'],
 8: ['[1A1a] Actividad principal producción de electricidad y calor',
  '[1A1a] Actividad principal producción de electricidad y calor'],
 9: ['[1A1b] Refinación del petróleo', '[1A1b] Refinación del petróleo'],
 10: ['[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía',
  '[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía'],
 11: ['[1A2] Industrias manufactura y de la construcción',
  '     1A1ci Fabricación de combustibles sólidos (coque de carbón)'],
 12: ['[1A2a] Hierro y acero', '1A1cii Otras Industrias de la energía  '],
 13: ['[1A2b] Metale

Para poder mejorar la visualización convertimos el diccionario en un dataframe:

In [10]:
mm = pd.DataFrame.from_dict(m, orient= 'index')
mm

,0,1
0,NaN,NaN
1,NaN,NaN
2,Potencial de calentamiento,Potencial de calentamiento
3,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),EMISIONES NETAS (Gg de CO2e)
...,...,...
182,-,[4D] Tratamiento y eliminación de aguas residu...
183,-,[4D1] Tratamiento y eliminación de aguas resid...
184,-,[4D2] Tratamiento y eliminación de aguas resid...
185,-,[4E] Otros


Imprimimos las primeras 20 y ultimas filas para intentar visualizar las diferencias:

In [11]:
mm.iloc[0:20,:] #primeros 20

,0,1
0,NaN,NaN
1,NaN,NaN
2,Potencial de calentamiento,Potencial de calentamiento
3,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),EMISIONES NETAS (Gg de CO2e)
5,[1] Energía,[1] Energía
6,[1A] Actividades de quema del combustible,[1A] Actividades de quema del combustible
7,[1A1] Industrias de la energía,[1A1] Industrias de la energía
8,[1A1a] Actividad principal producción de elect...,[1A1a] Actividad principal producción de elect...
9,[1A1b] Refinación del petróleo,[1A1b] Refinación del petróleo


In [12]:
mm.iloc[166:187,:] #ultimos 20

,0,1
166,[4C] Incineración y quema a cielo abierto de ...,[3C4] Emisiones directas de los N2O de lo...
167,[4C1] Incineración de residuos peligrosos indu...,[3C5] Emisiones indirectas de los N2O de ...
168,[4C2] Quema a cielo abierto de residuos sólidos,[3C6] Emisiones indirectas de los N2O de ...
169,[4D] Tratamiento y eliminación de aguas residu...,[3C7] Cultivo del arroz
170,[4D1] Tratamiento y eliminación de aguas resid...,[3D] Otros
171,[4D2] Tratamiento y eliminación de aguas resid...,[3D1] Productos de madera recolectada
172,[4E] Otros,[3D2] Otros (especificar)
173,EMISIONES NETAS (Gg de CO2e),[4] Residuos
174,NaN,[4A] Eliminación de residuos sólidos
175,Bunkers,[4A1] Sitios gestionados de eliminación de res...


Podemos ver que las filas varian ya que las bases de datos de 2020 y 2021 tiene un desglose adicional al de los años previos y aunado a eso se tienen más filas al final de las tablas en los años de 1990.

Para saber cuáles son las filas de desglose adicional en los años del 2020 y 2021 podemos usar un ciclo para obtener una lista de las filas:

In [13]:
demas = []
j=0
i=0
while (i<= 181 and j<=186):
    if type(df[2019].iloc[i,0]) != str:
        i=i+1
    if type(df[2020].iloc[j,0]) != str:
        j=j+1
        continue
    if df[2019].iloc[i,0].strip() == df[2020].iloc[j,0].strip():
        j= j+1
        i=i+1
    else:
        demas.append(j)
        j=j+1
         
demas

[11, 12, 17, 18, 19, 22, 23, 24, 25, 26, 35, 36, 37]

Lo que el ciclo hace es revisar la fila *i* para el dataframe del 2019 y la fila *j* del dataframe del 2020.

Primero valida que ambas sean del tipo *string* y de no ser así se pasa a la siguiente fila.

Si al comparar se encuentra que son iguales se pasa a la siguiente fila incrementando el valor de *i* y *j* simultaneamente.
Si se encuentra que no son iguales se agrega *j* a una lista y se pasa a la siguiente fila del dataframe de 2020 incrementando solo *j* para comprobar si esa corresponde a la fila actual del dataframe del 2019.

Además de eso se usa la función *strip* para evitar que haya diferencias debido a espacios vacios en las cadenas.

Una vez que se tiene una lista de las filas podemos quitarlas del dataframe:

In [14]:
temp = df[2020].transpose()

for i in demas:
    n= temp.pop(i)

regresa = temp.transpose()
regresa


,Unnamed: 0,2020,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,NaN,72.174862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


En esta ocasion se usa la transpuesta del dataframe para hacer posible el uso de la función *pop*.
Se pone la transpuesta en la variable **temp** y se usa un ciclo para remover las columnas(filas) que se marcaron en **demas**. 

Finalmente se vuelve a trasponer el dataframe con la ayuda de **regresa**.

Nótese que se le puede asignar un calor numérico a *pop* ya que al transponer el dataframe el nombre de las columnas son números.


Una vez que se tiene el dataframe corregido puede modificarse con la ayuda de la variable:

In [15]:
df[2020] = regresa
df[2020]

,Unnamed: 0,2020,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,NaN,72.174862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Usamos un método similar para el dataframe de 2021:

In [16]:
temp = df[2021].transpose()

for i in demas:
    n= temp.pop(i)

regresa1 = temp.transpose()
regresa1

,Unnamed: 0,2021,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),456265.876441,198041.61451,36315.62913,1974.597954,1670.82773,5.857883,34.841703,0.275913,8421.011818,...,0.29995,0.000003,0.107289,0.000002,2.96463,2.498574,714047.264077,714047.264077,NaN,66.371687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,25804.882349,5571.51224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31376.394589,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4930.620916,2781.021175,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7711.642091,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20874.261433,2790.491065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23664.752498,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [17]:
df[2021]=regresa1
df[2021]

,Unnamed: 0,2021,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),456265.876441,198041.61451,36315.62913,1974.597954,1670.82773,5.857883,34.841703,0.275913,8421.011818,...,0.29995,0.000003,0.107289,0.000002,2.96463,2.498574,714047.264077,714047.264077,NaN,66.371687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,25804.882349,5571.51224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31376.394589,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4930.620916,2781.021175,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7711.642091,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20874.261433,2790.491065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23664.752498,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Podemos usar otro método para quitar las filas sobrantes de los otros dataframes:

In [18]:
df[1990].drop(index=range(174,181))

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,0,0,0,0,0,36.517125,467413.123797,466720.011075,NaN,77.5572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,[4D] Tratamiento y eliminación de aguas residu...,NaN,9877.340649,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13367.176323,NaN,NaN,NaN
170,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN,NaN
171,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN,NaN
172,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Se usa la función *drop* con la indicación de que *index* (filas) tome los valores de 174 a 181.
Esto no cambia el dataframe de raíz ya que al momento de imprimirlo nuevamente, aun se tienen las filas finales:

In [19]:
df[1990]

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,0,0,0,0,0,36.517125,467413.123797,466720.011075,NaN,77.5572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Aviación internacional,2052.937,0.396,14.985,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2068.318,NaN,NaN,NaN
177,Marítimo internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Emisiones de CO2 por quema de biomasa,35955.863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35955.863,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Debido a esto se asignan a una nueva variable. En este caso se usa un diccionario **aux**:

In [20]:
aux = {}

for i in range(1990, 2020):
    aux[i]= df[i].drop(index=range(174,181))

aux[2019]

,Unnamed: 0,2019,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,NaN,65.581718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN,NaN
170,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN,NaN
171,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN,NaN
172,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Ahora con la ayuda del diccionario auxiliar se puede modificar **df**:

In [21]:
for i in range(1990,2020):
    df[i]=aux[i]

In [22]:
df[2019]

,Unnamed: 0,2019,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,NaN,65.581718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN,NaN
170,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN,NaN
171,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN,NaN
172,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Comprobamos una vez más que los tamaños sean iguales:

In [23]:
c = True

for i in range(1990,2021):
    if df[i].shape != df[i+1].shape:
        c = False
c

True

Una vez que se han homogenizado las filas se procede a corregir los nombres de las columnas que se encuentran distribuidos en 3 filas:

In [24]:
df[1990].iloc[0:2,:]

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN


Podemos ver que las primera fila, la fila de columnas es correspondiente a solamente el año:

In [25]:
df[1990].columns

Index([ 'Unnamed: 0',          1990,  'Unnamed: 2',  'Unnamed: 3',
        'Unnamed: 4',  'Unnamed: 5',  'Unnamed: 6',  'Unnamed: 7',
        'Unnamed: 8',  'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31'],
      dtype='object')

Esta fila se descartará ya que es no es indispensable para identificar el año.

La segunda fila lista gases de efecto invernadero y la tercera lista sirve como un desglose de los subtipos de algunos de estos:

In [26]:
l={}
for i in range(0,32):
    if type(df[1990].iloc[0,i]) == str:
        l[i]= df[1990].iloc[0,i]

l

{1: 'CO2',
 2: 'CH4',
 3: 'N2O',
 4: 'HFCs',
 20: 'PFCs',
 26: 'NF3',
 27: 'SF6',
 28: 'EMISIONES NETAS\nGg en CO2e',
 29: ' \nEMISIONES\n(sin 3B y 3D)\nGg en CO2e'}

In [27]:
l={}
for i in range(0,32):
    if type(df[1990].iloc[1,i]) == str:
        l[i]= df[1990].iloc[1,i]

l

{4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32',
 20: 'CF4',
 21: 'C2F6',
 22: 'C3F8',
 23: 'C4F6',
 24: 'c-C4F8',
 25: 'C5F8'}

Los tipos de gases que muestran subtipos son los HFCs y los PFCs ya que son los que se sobreponen con los gases de la fila 2. Para perservar lamayor información posible guardamos en un diccionario una lista de los gases de cada tipo:

In [28]:
HFC={}
for i in range(4,20):
    HFC[i]= df[1990].iloc[1,i]

HFC

{4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32'}

In [29]:
PCF={}
for i in range(20,26):
    PCF[i]= df[1990].iloc[1,i]

PCF

{20: 'CF4', 21: 'C2F6', 22: 'C3F8', 23: 'C4F6', 24: 'c-C4F8', 25: 'C5F8'}

Ahora juntamos ambas filas:

In [30]:
l={}
for i in range(0,32):
    if type(df[1990].iloc[0,i]) == str:
        l[i]= df[1990].iloc[0,i]
    if type(df[1990].iloc[1,i]) == str:
        l[i]= df[1990].iloc[1,i]

l

{1: 'CO2',
 2: 'CH4',
 3: 'N2O',
 4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32',
 20: 'CF4',
 21: 'C2F6',
 22: 'C3F8',
 23: 'C4F6',
 24: 'c-C4F8',
 25: 'C5F8',
 26: 'NF3',
 27: 'SF6',
 28: 'EMISIONES NETAS\nGg en CO2e',
 29: ' \nEMISIONES\n(sin 3B y 3D)\nGg en CO2e'}

Las columnas que quedan sin nombre son las columnas 0, 30 y 31.

La columna 30 aparenta estar vacía por lo que comprobamos. Usamos un ciclo para comprobar todos los datos de la columna:

In [31]:
c=True

for i in (0, len(df[1990])-1):
    if np.isnan(df[1990].iloc[i,30]) == False:
        c=False

c

True

Ya que se comprobó que es una columna vacía es posible desacernos de ella:

In [32]:
for i in range(1990,2022):
    p= df[i].pop(df[i].columns[30])


El ciclo recorre todos los dataframes quitando la columna 30. Se utiliza la función *columns* dentro de la función *pop* para que nos arroje el nombre de la columna ya que la función *pop* requiere el nombre de la columna.

Comprobamos que se haya eliminado la columna:

In [33]:
df[2021]

,Unnamed: 0,2021,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,11100,8900,1,9540,2,16100,23500,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),456265.876441,198041.61451,36315.62913,1974.597954,1670.82773,5.857883,34.841703,0.275913,8421.011818,...,3.233302,0.29995,0.000003,0.107289,0.000002,2.96463,2.498574,714047.264077,714047.264077,66.371687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,25804.882349,5571.51224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31376.394589,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4930.620916,2781.021175,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7711.642091,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20874.261433,2790.491065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23664.752498,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Ahora solo resta completar el nombre de la primera y última columna.

Se decide nombrar a la primera columna como "*Fuente*" y después de una revisión de las bases de datos originales se llega a la conclusión de que la última columna es nombrada "*Carbono negro (Gg)*".

También se decide corregir las columnas 28 y 29.

Para esto usamos nuevamente el diccionario de **l**:

In [34]:
l[0] = 'Fuentes'
l[30] = 'Carbono_negro(Gg)'
l[29] = 'EMISIONES(sin 3B y 3D)(Gg en CO2e)'
l[28] = 'EMISIONES_NETAS(Gg en CO2e)'

l


{1: 'CO2',
 2: 'CH4',
 3: 'N2O',
 4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32',
 20: 'CF4',
 21: 'C2F6',
 22: 'C3F8',
 23: 'C4F6',
 24: 'c-C4F8',
 25: 'C5F8',
 26: 'NF3',
 27: 'SF6',
 28: 'EMISIONES_NETAS(Gg en CO2e)',
 29: 'EMISIONES(sin 3B y 3D)(Gg en CO2e)',
 0: 'Fuentes',
 30: 'Carbono_negro(Gg)'}

Ahora podemos ordenar los nombres deseados de las columnas en una lista:

In [35]:
columnas = []
for i in range(0,31):
    columnas.append(l[i])

columnas

['Fuentes',
 'CO2',
 'CH4',
 'N2O',
 'HFC-23',
 'HFC-410A',
 'HFC-43-10mee',
 'HFC-125',
 'HFC-134',
 'HFC-134a',
 'HFC-404A',
 'HFC-407C',
 'HFC-507a',
 'HFC-152a',
 'HFC-227ea',
 'HFC-236fa',
 'HFC-365mfc/227ea',
 'HFC-365mfc',
 'HFC-245fa',
 'HFC-32',
 'CF4',
 'C2F6',
 'C3F8',
 'C4F6',
 'c-C4F8',
 'C5F8',
 'NF3',
 'SF6',
 'EMISIONES_NETAS(Gg en CO2e)',
 'EMISIONES(sin 3B y 3D)(Gg en CO2e)',
 'Carbono_negro(Gg)']

Ahora con la ayuda de esta lista se pueden corregir los nombres de las columnas y eliminar las filas antes señaladas:

In [36]:
df_col_correg = {}
tt = pd.DataFrame

for i in range(1990, 2022):
    tt = df[i].copy()
    tt.columns = columnas
    tt = tt.drop(0)
    tt = tt.drop(1)
    tt = tt.reset_index(drop=True)
    df_col_correg[i] = tt


El código anterior crea un nuevo diccionario para guardar los dataframes ya corregidos llamado **df_col_correg**. La asignación se realiza con la ayuda de una variable auxiliar llamada **tt** en la cuál se asigna una copia de los dataframes correspondientes, se corrigen los nombres de las columnas para después eliminar las filas extras y corregir la indexación.

Imprimimos algunos dataframe para observar que no haya problema:

In [37]:
df_col_correg[1990]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,11100,8900,1,9540,2,16100,23500,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
3,[1] Energía,287887.540936,15757.714204,2888.179063,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306533.434203,NaN,73.109323
4,[1A] Actividades de quema del combustible,277455.844583,2933.343959,2879.060741,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283268.249283,NaN,71.238034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,[4D] Tratamiento y eliminación de aguas residu...,NaN,9877.340649,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13367.176323,NaN,NaN
168,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN
169,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN
170,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [38]:
df_col_correg[2020]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,11100,8900,1,9540,2,16100,23500,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,2.696136,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,72.174862
3,[1] Energía,414480.670039,34441.977884,2130.668697,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451053.31662,NaN,64.050795
4,[1A] Actividades de quema del combustible,398693.932566,3495.915494,2119.725053,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404309.573112,NaN,55.570891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN
168,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN
169,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN
170,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Al revisar los nuevos dataframes se observa lo siguiente:
* La fila 0 (Potencial de calentamiento) parece ser igual en todos los dataframes ya que corresponde a una característica propia de cada gas
* La fila 1 parece estar vacía
* Las filas 2 y 171 parecen ser iguales

Comprobamos que la fila 0 es igual en todos los dataframes:

In [39]:
c=True
for i in range(1990, 2020):
    for j in range(1,31):
        if df_col_correg[i].iloc[0,j] != df_col_correg[i+1].iloc[0,j]:
            c= False
            break

c

False

Ya que el resultado es 'False' imprimimos *i* y *j* para ver donde se da la diferencia:

In [40]:
print(i, j)

2019 28


Revisamos los datos:

In [41]:
df_col_correg[i+1].iloc[0,j]

nan

Ya que en los datos son del tipo '*nan*' es posible que la naturaleza de los datos ocasionen la diferencia por lo que validamos nuevamente esta vez considerando este tipo de datos:

In [42]:
c=True
for i in range(1990, 2020):
    for j in range(1,31):
        if (np.isnan(df_col_correg[i].iloc[0,j]) and np.isnan(df_col_correg[i+1].iloc[0,j])) == True:
            continue
        if df_col_correg[i].iloc[0,j] != df_col_correg[i+1].iloc[0,j]:
            c= False
            break

c

True

Ya que se validó que la fila es igual en todas de momento se decide quitarla y conservar la información en una variable distinta:

In [43]:
Potencial_de_calentamiento= {}
for i in range(0,31):
    Potencial_de_calentamiento[df_col_correg[1990].columns[i]] = df_col_correg[1990].iloc[0,i]

Potencial_de_calentamiento

{'Fuentes': '  Potencial de calentamiento',
 'CO2': 1,
 'CH4': 28,
 'N2O': 265,
 'HFC-23': 12400,
 'HFC-410A': 1924,
 'HFC-43-10mee': 1650,
 'HFC-125': 3170,
 'HFC-134': 1120,
 'HFC-134a': 1300,
 'HFC-404A': 3943,
 'HFC-407C': 1624,
 'HFC-507a': 3985,
 'HFC-152a': 138,
 'HFC-227ea': 2640,
 'HFC-236fa': 8060,
 'HFC-365mfc/227ea': 982,
 'HFC-365mfc': 804,
 'HFC-245fa': 858,
 'HFC-32': 677,
 'CF4': 6630,
 'C2F6': 11100,
 'C3F8': 8900,
 'C4F6': 1,
 'c-C4F8': 9540,
 'C5F8': 2,
 'NF3': 16100,
 'SF6': 23500,
 'EMISIONES_NETAS(Gg en CO2e)': nan,
 'EMISIONES(sin 3B y 3D)(Gg en CO2e)': nan,
 'Carbono_negro(Gg)': nan}

Una vez que la información se guardó en un diccionario separado, removemos la fila de todos los dataframes:

In [44]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].drop(0)

In [45]:
df_col_correg[2019]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,2.629593,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,65.581718
3,[1] Energía,444504.90145,21170.54218,2216.930109,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,467892.373739,NaN,54.963710
4,[1A] Actividades de quema del combustible,432110.270818,2821.764602,2182.895974,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437114.931394,NaN,48.860462
5,[1A1] Industrias de la energía,202816.918337,173.965545,303.316786,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203294.200668,NaN,8.039951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN
168,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN
169,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN
170,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Reseteamos los indices para que no haya confusiones:

In [46]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].reset_index(drop=True)

In [47]:
df_col_correg[2019]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,2.629593,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,65.581718
2,[1] Energía,444504.90145,21170.54218,2216.930109,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,467892.373739,NaN,54.963710
3,[1A] Actividades de quema del combustible,432110.270818,2821.764602,2182.895974,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437114.931394,NaN,48.860462
4,[1A1] Industrias de la energía,202816.918337,173.965545,303.316786,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203294.200668,NaN,8.039951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN
167,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN
168,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN
169,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Ahora procedemos a verificar que la fila 1 este vacía:

In [48]:
c= False
for i in range(0,31):
    if np.isnan(df_col_correg[1990].iloc[0,i]) == False:
        c = True

c

False

Ya que comprobamos que está vacía podemos eliminarla de los dataframes:

In [49]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].drop(0)
    df_col_correg[i]=df_col_correg[i].reset_index(drop=True)
    

In [50]:
df_col_correg[2020]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,2.696136,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,72.174862
1,[1] Energía,414480.670039,34441.977884,2130.668697,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451053.31662,NaN,64.050795
2,[1A] Actividades de quema del combustible,398693.932566,3495.915494,2119.725053,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404309.573112,NaN,55.570891
3,[1A1] Industrias de la energía,174609.239141,139.906396,213.414841,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174962.560377,NaN,5.401026
4,[1A1a] Actividad principal producción de elect...,147501.152024,125.631395,197.19366,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147823.977079,NaN,4.828733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN
166,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN
167,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN
168,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Ahora comprobamos si la fila 0 y 169 son iguales para todos los dataframes:

In [51]:
c=True
for i in range(1990,2022):
    for j in range(1,31):
        if (np.isnan(df_col_correg[i].iloc[0,j]) and np.isnan(df_col_correg[i].iloc[169,j])) == True:
            continue
        if df_col_correg[i].iloc[0,j] != df_col_correg[i].iloc[169,j]:
            c=False
            break

c

False

Vemos que no son iguales así que comprobamos los vañores de *i* y *j* así como los valores que se marcan como diferentes

In [52]:
print(i,j)

2021 1


In [53]:
print(df_col_correg[i].iloc[0,j],df_col_correg[i].iloc[169,j])

456265.8764410899 456265.87644108996


Vemos que las diferencias son bastante pequeñas. Comprobamos que este sea el caso para todos los valores y dataframes:

In [54]:
dif= []

for i in range(1990,2022):
    aux=[]
    for j in range(1,31):
        aux.append(df_col_correg[1990].iloc[0,j] - df_col_correg[1990].iloc[169,j])
    
    dif.append(np.nanmean(aux))


El ciclo calcula las diferencias entre cada fila y lo guarda en una lista auxiliar para después obtener el promedio de las diferencias de cada dataframe y guardarla en la lista **dif**. Ahora podemos ver cuál es el valor máximo de esa lista y verificar si es una diferencia significativa:

In [55]:
np.max(dif)

4.014811471781035e-12

Ya que el valor es muy pequeño no se consideran diferencias significativas por lo que se elimina una de ellas.

In [56]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].drop(169)
    df_col_correg[i]=df_col_correg[i].reset_index(drop=True)

In [57]:
df_col_correg[1990]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
1,[1] Energía,287887.540936,15757.714204,2888.179063,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306533.434203,NaN,73.109323
2,[1A] Actividades de quema del combustible,277455.844583,2933.343959,2879.060741,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283268.249283,NaN,71.238034
3,[1A1] Industrias de la energía,107765.12174,92.051436,187.539901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108044.713077,NaN,14.152478
4,[1A1a] Actividad principal producción de elect...,70302.750576,62.745182,139.850236,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70505.345994,NaN,11.656345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,[4C2] Quema a cielo abierto de residuos sólidos,402.492892,801.997327,175.161504,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379.651724,NaN,1.354876
165,[4D] Tratamiento y eliminación de aguas residu...,NaN,9877.340649,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13367.176323,NaN,NaN
166,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN
167,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN


Para evitar futuros errores se eliminan los espacios al inicio y final de las cadenas de texto de la columna 0:

In [58]:
for i in range(1990,2022):
    for j in range(0,169):
        df_col_correg[i].iloc[j,0] = df_col_correg[i].iloc[j,0].strip()

df_col_correg[2019].iloc[:,0]

0                           EMISIONES NETAS (Gg de CO2e)
1                                            [1] Energía
2              [1A] Actividades de quema del combustible
3                         [1A1] Industrias de la energía
4      [1A1a] Actividad principal producción de elect...
                             ...                        
164      [4C2] Quema a cielo abierto de residuos sólidos
165    [4D] Tratamiento y eliminación de aguas residu...
166    [4D1] Tratamiento y eliminación de aguas resid...
167    [4D2] Tratamiento y eliminación de aguas resid...
168                                           [4E] Otros
Name: Fuentes, Length: 169, dtype: object

Ya que algunas filas son correspondientes a una subclase de otras se considera conveniente saber en que nivel jerarquico se encuentra cada una de ellas. 

Para esto se usará el hecho de que en la mayoría de los casos la subdivisión está dada entre corchetes (**[ ]**).

La base tiene 4 grandes categorías, identificadas con los números del 1 al 4, a las cuales se les agregan letras y números para identificar las subcategorías correspondientes a cada una. Es decir la categoría *[1]* se divide a su vez en *[1A]* y *[1B]*. A su vez *[1A]* se divide en *[1A1]*, *[1A2]*, *[1A3]* y *[1A4]*. Después *[1A1]* se divide en *[1A1a]*, *[1A1b]* y *[1A1c]*. Finalmente al llegar a un quinto nivel el desglose se quita de los corchetes, por ejemplo, la categoría *[1A1c]* se divide en *1A1ci* y *1A1cii*.

Podemos usar la cantidad de caracteres dentro de los corchetes para encontrar el nivel de clasificación as filas:

In [77]:
clasif = {}

for i in range(0,169):
 #clasif.append(df_col_correg[1990].iloc[i,0].find(']')-df_col_correg[1990].iloc[i,0].find('['))
 clasif[df_col_correg[1990].iloc[i,0]] = df_col_correg[1990].iloc[i,0].find(']')-df_col_correg[1990].iloc[i,0].find('[')

clasif

{'EMISIONES NETAS (Gg de CO2e)': 0,
 '[1] Energía': 2,
 '[1A] Actividades de quema del combustible': 3,
 '[1A1] Industrias de la energía': 4,
 '[1A1a] Actividad principal producción de electricidad y calor': 5,
 '[1A1b] Refinación del petróleo': 5,
 '[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía': 5,
 '[1A2] Industrias manufactura y de la construcción': 4,
 '[1A2a] Hierro y acero': 5,
 '[1A2b] Metales no ferrosos': 5,
 '[1A2c] Sustancias químicas': 5,
 '[1A2d] Pulpa, papel e imprenta': 5,
 '[1A2e] Procesamiento de alimentos, bebidas y tabaco': 5,
 '[1A2f] Minerales no metálicos': 5,
 '[1A2g] Equipo de transporte': 5,
 '[1A2h] Maquinaria': 5,
 '[1A2i] Minería (con excepción de combustibles) y cantería': 5,
 '[1A2j] Madera y productos de la madera': 5,
 '[1A2k] Construcción': 5,
 '[1A2l] Textiles y cueros': 5,
 '[1A2m] Industria no especificada': 5,
 '[1A3] Transporte': 4,
 '[1A3a] Aviación civil': 5,
 '[1A3b] Autotransporte': 5,
 '[1A3c] Ferrocarriles': 5,


Del diccionario anterior vemos que los resutados están relacionados con el nivel de clasificación:
- *[1]* primer nivel: 2
- *[1A]* segundo nivel: 3
- *[1A1]* tercer nivel: 4
- *[1A1a]* cuarto nivel: 5

Se tienen también resultados de 6 y 7 pero estos pertenecen a un quinto nivel:
- [1B1ai]
- [1B1aii]

Los valores de 0 se imprimen para saber si pertenecen a algun nivel de clasificación:

In [86]:
for i in clasif.keys():
    if clasif[i] == 0:
     print (clasif[i], i)

0 EMISIONES NETAS (Gg de CO2e)
0 1B2ai Venteo petróleo
0 1B2aii Quemado petróleo
0 1B2aiii Otras fugitivas petróleo
0 1B2bi Venteo gas natural
0 1B2bii Quemado gas natural
0 1B2biii Otras fugitivas gas natural


Como podemos ver todos excepcto el correspondiente a la *fila 0* corresponden a clasificación de quinto nivel.